MAIN SCRIPT

In [296]:
import numpy as np
from helpers import *
from clean_data import clean_data
from visualize_features import visualize_features
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [300]:
# Load Data
path_dataset = "train.csv"
yb, input_data, ids = load_csv_data(path_dataset)

In [ ]:
# Data Cleaning
tx0, tx1, tx2, tx3, y0, y1, y2, y3 = clean_data(input_data,yb) # Note: Didn't use ids, no use

In [301]:
# Feature Visualization
features = np.array([1,4,5]) # Features to compare
visualize_features(tx0,y0,features,'Feature_comparision')